1.load the data and libraries

2.clean the data

3.tokenize the data

4.convert to sequence

5.input seqquence and output sequence

6.create a sequential model

7.LSTM layer

8.compile the model

9.fit the model

10.evaluate the model

In [ ]:
from random import randint
from pickle import load,dump
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dropout # Change 'dropout' to 'Dropout'
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from numpy import array
import string
import urllib # Correct the typo from 'urlib' to 'urllib'


In [ ]:
response=urllib.request.urlopen('https://raw.githubusercontent.com/insaid2018/DeepLearning/master/Data/republic_clean.txt?sm_au=iVV10f0f2kPt2J07')
doc=response.read().decode('utf-8')

In [ ]:
print(doc[:1000])




















appear


In [ ]:
def clean_doc(doc):
  doc=doc.replace('--',' ')
  tokens=doc.split()
  table=str.maketrans('','',string.punctuation)
  tokens=[w.translate(table) for w in tokens]
  tokens=[word for word in tokens if word.isalpha()]
  tokens=[word.lower() for word in tokens]
  return tokens

In [ ]:
tokens = clean_doc(doc)

In [ ]:
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
#

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [ ]:
length=50+1
sequences=list()
for i in range(length,len(tokens)):
  seq=tokens[i-length:i]
  line=' '.join(seq)
  sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 118633


In [ ]:
sequences

['book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was',
 'i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted',
 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with',
 'went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what man

In [ ]:
def save_doc(lines,filename):
  data='\n'.join(lines)
  file=open(filename,'w')
  file.write(data)
  file.close()

In [ ]:
out_filename='republic_sequences.txt'
save_doc(sequences,out_filename)

In [ ]:
def load_doc(filename):
  file=open(filename,'r')
  text=file.read()
  file.close()
  return text
#

In [ ]:
in_filename='republic_sequences.txt'
doc=load_doc(in_filename)
lines=doc.split('\n')

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
sequences = array(sequences)

In [ ]:
sequences

array([[1046,   11,   11, ...,  151,   11,   57],
       [  11,   11, 1045, ...,   11,   57, 1147],
       [  11, 1045,  329, ...,   57, 1147,   35],
       ...,
       [ 382,  467,    4, ...,  414,   13,   21],
       [ 467,    4,   33, ...,   13,   21,   23],
       [   4,   33,   79, ...,   21,   23,   85]])

In [ ]:
sequences.shape

(118633, 51)

In [ ]:
X,y = sequences[:,:-1],sequences[:,-1]

In [ ]:
y[1]

1147

In [ ]:
X.shape

(118633, 50)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 7410


In [ ]:
X.shape

(118633, 50)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=X.shape[1]))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - accuracy: 0.0617 - loss: 6.4642
Epoch 2/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.1039 - loss: 5.7147
Epoch 3/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.1244 - loss: 5.4759
Epoch 4/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.1432 - loss: 5.2940
Epoch 5/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.1539 - loss: 5.1791
Epoch 6/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1590 - loss: 5.0658
Epoch 7/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1662 - loss: 4.9710
Epoch 8/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.1732 - loss: 4.8898
Epoch 9/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1761 - loss: 4.8227
Epoch 10/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.1804 - loss: 4.7505
Epoch 11/100
927/927 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1820 - loss: 4.6811
Epoch 12/100
927/92

In [ ]:
import numpy as np

In [ ]:
def genarate_seq (model,tokenizer,max_length,seed_text,n_words):
  in_text=seed_text
  for _ in range(n_words):
    encoded=tokenizer.texts_to_sequences([in_text])[0]
    encoded=pad_sequences([encoded], maxlen=max_length, padding='pre')
    yhat=model.predict(encoded, verbose = 0)
    out_word=''
    for word, index in tokenizer.word_index.items():
      if index == yhat.argmax():
        out_word=word
        break
    in_text+=' '+out_word
  return in_text #Fixed indentation

In [ ]:
seq_length = len(lines[0].split())-1

In [ ]:
seed_text = lines[randint(0,len(lines))]
print("seed_text:" + '\n')
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print("generated_text:" + '\n')
print(generated)

seed_text:

has the rim broadest and the seven inner whorls are narrower in the following proportions the sixth is next to the first in size the fourth next to the sixth then comes the eighth the seventh is fifth the fifth is sixth the third is seventh last and eighth comes the

generated_text:

other of all things but whole tops when they spin round with them and that the soul which is the inference and the other of the soul and the other of mankind will not be quelled until he inherited and injured other purpose they were to be told to compose
